# [DeepTraffic](https://selfdrivingcars.mit.edu/deeptrafficjs/)

### [Школа GoTo](https://goto.msk.ru)
[Емельяненко Дмитрия](https://github.com/TIXFeniks) <br>
[Творожков Андрей](https://tvorog.me)

DeepTraffic - это симуляция типично нагруженной дороги. Ваша задача разработать нейронную сеть, которая поможет ускорить трафик на загруженных дорогах.

![](https://selfdrivingcars.mit.edu/wordpress/wp-content/uploads/2017/01/Screen-Shot-2017-01-03-at-16.02.05-1024x640.png)

Ваша нейронная сеть получает контроль над машиной (обозначена красным цветом) и должна обчиться управлять ей максимально эффективно, чтобы ехать как можно быстрее. Машина уже снабжена системой безопасности, поэтому вы можете не заботиться о столкновениях с другими машинами. Сеть может влиять только на изменение скорости и изменение линий. 


## Разберём стандартный код

Есть 3 параметра ответственные за количество клеток передаваемых 

```
lanesSide = 0; // Количество лийи вокруг *радиус*
patchesAhead = 1; // Количество клеток впереди
patchesBehind = 0; // Количество клеток позади
```

Вот несколько примеров изменений этих параметров:

![](./screens/1.png)

<br><br>
## Идём дальше
<br><br>
```
// Количество итераций при нажатии кнопки начать обучение
trainIterations = 10000; 

// (Количество линий * 2 (справа и слева) + 1 (по середине)) * (количество клеток впереди + количество клето позади)
var num_inputs = (lanesSide * 2 + 1) * (patchesAhead + patchesBehind);

// Бездействовать = 0;
// Ускорится = 1;
// Замедлиться = 2;
// Повернуть налево = 3;
// Повернуть направо = 4;

// Количество действи
var num_actions = 5;

// Количество раз которое мы запоминаем
var temporal_window = 3;

// Количество нейронов на вход
var network_size = num_inputs * temporal_window + num_actions * temporal_window + num_inputs;
```

<br><br>
# Нейронная сеть!
![](https://uccexpress.ie/wp-content/uploads/2017/09/6360498178139720611073070814_Rick-and-Morty.jpg)
<br><br>

```
// Массив со слоями нейронной сети
var layer_defs = [];

// Входной слой
layer_defs.push({
    type: 'input',
    out_sx: 1,
    out_sy: 1,
    out_depth: network_size
});

// Полносвязный слой
layer_defs.push({
    type: 'fc',
    num_neurons: 1,
    activation: 'relu'
});

// Выходной слой, который предсказывает вероятности 
layer_defs.push({
    type: 'regression',
    num_neurons: num_actions
});
```

## Коэффициент скорости обучения
Большие значения (0,7 – 1) будут соответствовать большому значению шага коррекции. При этом алгоритм будет работать быстрее (т.е. для поиска минимума функции ошибки потребуется меньше шагов), однако может снизиться точность настройки на минимум, что потенциально увеличит ошибку обучения. Малые значения коэффициента (0,1 – 0,3) соответствуют меньшему шагу коррекции весов. При этом число шагов (или эпох), требуемое для поиска оптимума, как правило, увеличивается, но возрастает и точность настройки на минимум, что потенциально уменьшает ошибку обучения. На практике коэффициент скорости обучения обычно подбирают экспериментально.

```
var tdtrainer_options = {
    // Коэффициент скорости обучения
    learning_rate: 0.001,
    
    // Количество объектов, которые мы передаём за 1 итерацию обучения
    batch_size: 64,
    l2_decay: 0.01,
    momentum: 0.0,
};
```

```
// Объект в котором будем хранить настройки
var opt = {};

// Описывалось выше
opt.temporal_window = temporal_window;

// size of experience replay memory
opt.experience_size = 3000;

// number of examples in experience replay memory before we begin learning
opt.start_learn_threshold = 500;

// gamma is a crucial parameter that controls how much plan-ahead the agent does. In [0,1]
opt.gamma = 0.7;

// number of steps we will learn for
opt.learning_steps_total = 10000;

// how many steps of the above to perform only random actions (in the beginning)?
opt.learning_steps_burnin = 1000;

// what epsilon value do we bottom out on? 0.0 => purely deterministic policy at end
opt.epsilon_min = 0.0;

// what epsilon to use at test time? (i.e. when learning is disabled)
opt.epsilon_test_time = 0.0;

// Наши слои
opt.layer_defs = layer_defs;

// Настройки нейронки
opt.tdtrainer_options = tdtrainer_options;

// Чёрная коробка
brain = new deepqlearn.Brain(num_inputs, num_actions, opt);
```

# И наконец - последняя функция которая делает магию
![](http://s9.pikabu.ru/post_img/2016/12/12/8/og_og_1481547098267285367.jpg)

```
// Функция, которая вызывается при обучении
learn = function (state, lastReward) {
    // Тут происходит вся магия
    brain.backward(lastReward);
    
    // Получаем action
    var action = brain.forward(state);
       
    // Функции которые рисуют статистику
    draw_net();
    draw_stats();
       
    // Возвращаем action
    return action;
}
```

## Код
### Скопируй и вставь
```
lanesSide = 0; // Количество лийи вокруг *радиус*
patchesAhead = 1; // Количество клеток впереди
patchesBehind = 0; // Количество клеток позади

// Количество итераций при нажатии кнопки начать обучение
trainIterations = 10000; 

// (Количество линий * 2 (справа и слева) + 1 (по середине)) * (количество клеток впереди + количество клето позади)
var num_inputs = (lanesSide * 2 + 1) * (patchesAhead + patchesBehind);

// Бездействовать = 0;
// Ускорится = 1;
// Замедлиться = 2;
// Повернуть налево = 3;
// Повернуть направо = 4;

// Количество действи
var num_actions = 5;

// Количество раз которое мы запоминаем
var temporal_window = 3;

// Количество нейронов на вход
var network_size = num_inputs * temporal_window + num_actions * temporal_window + num_inputs;

// Массив со слоями нейронной сети
var layer_defs = [];

// Входной слой
layer_defs.push({
    type: 'input',
    out_sx: 1,
    out_sy: 1,
    out_depth: network_size
});

// FC (i.e. fully-connected)
layer_defs.push({
    type: 'fc',
    num_neurons: 1,
    activation: 'relu'
});

// Выходной слой, который предсказывает вероятности 
layer_defs.push({
    type: 'regression',
    num_neurons: num_actions
});

var tdtrainer_options = {
    // Коэффициент скорости обучения
    learning_rate: 0.001,

    // Количество объектов, которые мы передаём за 1 итерацию обучения
    batch_size: 64,

    l2_decay: 0.01,
    momentum: 0.0,
};

// Объект в котором будем хранить настройки
var opt = {};

// Описывалось выше
opt.temporal_window = temporal_window;

// size of experience replay memory
opt.experience_size = 3000;

// number of examples in experience replay memory before we begin learning
opt.start_learn_threshold = 500;

// gamma is a crucial parameter that controls how much plan-ahead the agent does. In [0,1]
opt.gamma = 0.7;

// number of steps we will learn for
opt.learning_steps_total = 10000;

// how many steps of the above to perform only random actions (in the beginning)?
opt.learning_steps_burnin = 1000;

// what epsilon value do we bottom out on? 0.0 => purely deterministic policy at end
opt.epsilon_min = 0.0;

// what epsilon to use at test time? (i.e. when learning is disabled)
opt.epsilon_test_time = 0.0;

// Наши слои
opt.layer_defs = layer_defs;

// Настройки нейронки
opt.tdtrainer_options = tdtrainer_options;

// Чёрная коробка
brain = new deepqlearn.Brain(num_inputs, num_actions, opt);

// Функция, которая вызывается при обучении
learn = function (state, lastReward) {
    // Тут происходит вся магия
    brain.backward(lastReward);

    // Получаем action
    var action = brain.forward(state);

    // Функции которые рисуют статистику
    draw_net();
    draw_stats();

    // Возвращаем action
    return action;
}
```

# Ура!

## Что делать дальше?
1. Можно посмотреть в эти же тетрадки дома и разобраться более детально. [Вот](https://github.com/tvorogme/digitalfest) репозиторий!
2. Прочитай [блог школы GoTo](https://habrahabr.ru/company/goto/blog/339050/), рассказывающий, с чего начинать изучать анализ данных и машинное обучение.
3. Когда ты наберёшься знаний и тебе захочется проверить свои силы, попробуй поучаствовать в соревнованиях на [kaggle](https://www.kaggle.com/competitions)

4. Когда ты научишься самостоятельно обучать нейронные сети, CPU для вычислений начнёт не хватать. Подумай о покупке GPU. Подойдёт любая CUDA-совместимая видеокарта, но чем мощнее - тем лучше
<br><br>
![](screens/zmubBCUZwBg.jpg)